In [51]:
import torch


In [52]:
inputs = torch.nn.Embedding(4,8)


In [53]:
inputs = inputs.weight
inputs

Parameter containing:
tensor([[ 0.5442, -0.3583,  0.4284, -0.5506, -0.6946, -2.0262, -1.6451, -0.7431],
        [-0.3511, -1.7066, -0.2856,  0.6253, -0.8071,  0.9050, -0.0854,  0.5278],
        [ 1.1195,  0.3508,  1.0625, -0.1631,  1.9232,  0.1876, -1.9495,  0.9537],
        [ 0.1406, -0.7225,  0.2274,  0.8747, -2.0024,  2.4018, -0.2076, -1.6268]],
       requires_grad=True)

In [54]:
inputs = inputs.data
inputs

tensor([[ 0.5442, -0.3583,  0.4284, -0.5506, -0.6946, -2.0262, -1.6451, -0.7431],
        [-0.3511, -1.7066, -0.2856,  0.6253, -0.8071,  0.9050, -0.0854,  0.5278],
        [ 1.1195,  0.3508,  1.0625, -0.1631,  1.9232,  0.1876, -1.9495,  0.9537],
        [ 0.1406, -0.7225,  0.2274,  0.8747, -2.0024,  2.4018, -0.2076, -1.6268]])

In [58]:
#create weight matrices and set dimensions
d_in = 8
d_out = 6
W_q = torch.nn.Parameter(torch.rand((d_in,d_out)), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand((d_in,d_out)), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand((d_in,d_out)), requires_grad=False)

In [59]:

#choose input vector and transform it into our query vector using W_q

query = inputs[2] @W_q
query



tensor([1.9045, 1.8137, 2.1289, 2.3067, 2.6276, 0.5842])

In [60]:
#calcualte attention scores using keys generated by w_k
keys = inputs @ W_k
values = inputs @ W_v
print("Keys:", keys)
print("Values:", values)

Keys: tensor([[-2.3055, -3.5374, -3.0642, -2.1090, -1.8208, -1.4988],
        [-0.3400, -0.3018, -0.0149, -0.8319, -2.2124, -1.4911],
        [ 1.5205,  1.4319,  2.3963,  0.9467,  1.4426,  1.3657],
        [-1.8230, -1.0397,  0.7561, -0.0270, -1.7616, -0.3677]])
Values: tensor([[-2.3307, -2.1374, -1.8553, -1.1769, -2.9780, -3.3913],
        [-0.3609, -0.0309, -0.0068, -1.1789,  0.7904, -0.6430],
        [ 2.0029,  0.8580,  1.2310,  2.1924,  2.7329,  2.0493],
        [-0.9863, -0.1657,  1.3662, -1.3847,  0.9532, -1.9188]])


In [34]:
attention_scores = query @ keys.T
attention_scores

tensor([7.0591, 9.4606, 7.5099, 1.1274])

In [61]:
attention_weights = torch.softmax( attention_scores / keys.shape[-1]**0.5, dim=-1 )
attention_weights

tensor([0.2018, 0.5378, 0.2425, 0.0179])

In [62]:
attentions_weights.sum()

tensor(1.0000)

In [63]:
context_vector = attentions_weights @ values
context_vector

tensor([-0.1963, -0.2427, -0.0549, -0.3646,  0.5041, -0.5674])

In [64]:
import torch.nn as nn

In [65]:
class SimpleAttention( nn.Module ):
    def __init__(self, d_in, d_out):
        super().__init__()
        #create weight matrices
        self.W_q = nn.Parameter( torch.randn( (d_in, d_out), requires_grad=False ) )
        self.W_k = nn.Parameter( torch.randn( (d_in, d_out), requires_grad=False ) )
        self.W_v = nn.Parameter( torch.randn( (d_in, d_out), requires_grad=False ) )

    # x = embedding vectors (inputs)
    def forward(self, x):
        queries = x @ self.W_q
        keys = x @ self.W_k
        values = x @ self.W_v
        scores = queries @ keys.T
        weights = torch.softmax( scores / keys.shape[-1]**0.5, dim = -1 )
        context = weights @ values
        return context; 
        

In [66]:
simple = SimpleAttention(d_in = 8, d_out = 6,)

In [67]:
simple.W_k

Parameter containing:
tensor([[-0.0431,  0.0146, -1.2971, -0.0171,  0.5555, -0.1975],
        [-1.4581, -1.0632,  0.9727, -0.2306,  0.6323, -1.5716],
        [ 1.8000,  0.6147,  0.3825, -1.7541, -0.5908,  1.5765],
        [-1.1288,  1.3772,  0.3967, -1.1890,  1.9831,  1.4136],
        [-0.6202, -0.6380,  1.3297, -0.9203,  0.0260,  1.4321],
        [-1.0993,  0.9146,  0.1289, -0.6392, -1.6315,  0.2007],
        [-0.8788, -0.5630, -1.0449, -1.1407,  0.6796,  0.4080],
        [-1.9095,  0.2027, -0.0195,  0.3786,  0.5853,  0.5494]],
       requires_grad=True)

In [68]:
context_vectors = simple(inputs)
context_vectors

tensor([[-3.5676,  0.1781, -1.1646,  3.9914, -0.4402, -2.4747],
        [-0.0714, -4.0302,  2.7484, -3.6080, -4.0779,  0.7696],
        [-2.8627, -0.3220, -0.8772,  3.0581, -0.9525, -1.7089],
        [ 0.8136, -4.3467,  3.2426, -5.0195, -3.7475,  1.0389]],
       grad_fn=<MmBackward0>)

In [69]:
#second version but its linear not rand

class SimpleAttentionv2( nn.Module ):
    def __init__(self, d_in, d_out):
        super().__init__()
        #create weight matrices
        self.W_q = nn.Linear( d_in, d_out, bias=False )
        self.W_k = nn.Linear( d_in, d_out, bias=False )
        self.W_v = nn.Linear( d_in, d_out, bias=False )


    # x = embedding vectors (inputs)
    def forward( self, x ):
        queries = self.W_q( x )
        keys = self.W_k( x )
        values = self.W_v( x )
        scores = queries @ keys.T
        weights = torch.softmax( scores / keys.shape[-1]**0.5, dim = -1 )
        context = weights @ values
        return context

In [70]:
simple = SimpleAttentionv2(d_in = 8, d_out=6)


In [71]:
context_vectors = simple(inputs)
context_vectors

tensor([[ 0.5182, -0.4823, -0.2079, -0.1227, -0.2912,  0.0909],
        [ 0.6335, -0.0115,  0.0521, -0.3479, -0.1974, -0.1220],
        [ 0.4956, -0.4126, -0.1995, -0.1590, -0.2194,  0.0902],
        [ 0.5541,  0.0658,  0.1989, -0.3611, -0.1316, -0.0659]],
       grad_fn=<MmBackward0>)

In [72]:
weights = torch.rand(inputs.shape[0], inputs.shape[0])

In [73]:
weights

tensor([[0.4148, 0.3903, 0.3606, 0.7087],
        [0.5272, 0.3624, 0.2373, 0.4305],
        [0.3355, 0.2843, 0.4658, 0.1366],
        [0.1564, 0.0350, 0.8193, 0.7746]])

In [75]:
weights.sum(dim=-1)

tensor([1.8743, 1.5573, 1.2223, 1.7852])

In [76]:
simple_mask = torch.tril(torch.ones(weights.shape[0],weights.shape[0]))
simple_mask

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [77]:
masked_weights = weights*simple_mask
masked_weights

tensor([[0.4148, 0.0000, 0.0000, 0.0000],
        [0.5272, 0.3624, 0.0000, 0.0000],
        [0.3355, 0.2843, 0.4658, 0.0000],
        [0.1564, 0.0350, 0.8193, 0.7746]])

In [78]:
masked_weights.sum(dim= -1)

tensor([0.4148, 0.8896, 1.0857, 1.7852])

In [79]:
row_sums = masked_weights.sum(dim=  -1, keepdim=True)
row_sums

tensor([[0.4148],
        [0.8896],
        [1.0857],
        [1.7852]])

In [80]:
masked_weights = masked_weights/row_sums
masked_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5927, 0.4073, 0.0000, 0.0000],
        [0.3090, 0.2619, 0.4291, 0.0000],
        [0.0876, 0.0196, 0.4589, 0.4339]])

In [82]:
mask = torch.triu( torch.ones(weights.shape[0], weights.shape[0]), diagonal=1 )
mask

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])

In [84]:
mask.bool()

tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

In [85]:
weights

tensor([[0.4148, 0.3903, 0.3606, 0.7087],
        [0.5272, 0.3624, 0.2373, 0.4305],
        [0.3355, 0.2843, 0.4658, 0.1366],
        [0.1564, 0.0350, 0.8193, 0.7746]])

In [86]:
weights = weights.masked_fill( mask.bool(), -torch.inf)
weights

tensor([[0.4148,   -inf,   -inf,   -inf],
        [0.5272, 0.3624,   -inf,   -inf],
        [0.3355, 0.2843, 0.4658,   -inf],
        [0.1564, 0.0350, 0.8193, 0.7746]])

In [87]:
masked_weights = torch.softmax(weights, dim=-1)
masked_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5411, 0.4589, 0.0000, 0.0000],
        [0.3237, 0.3075, 0.3688, 0.0000],
        [0.1760, 0.1559, 0.3415, 0.3266]])

In [88]:
masked_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000])

In [89]:
#drop out a technique to prevent overfitting and making a model too good for one task makes it super poor at anything else, dropout fixes this
dropout = nn.Dropout(.5)

In [90]:
dropout(inputs)

tensor([[ 1.0884, -0.0000,  0.8569, -0.0000, -0.0000, -0.0000, -3.2903, -1.4862],
        [-0.7022, -0.0000, -0.0000,  0.0000, -0.0000,  1.8101, -0.1707,  0.0000],
        [ 2.2391,  0.7016,  0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  1.9074],
        [ 0.0000, -0.0000,  0.4547,  1.7494, -0.0000,  4.8036, -0.4152, -0.0000]])

In [91]:
batches = torch.stack((inputs,inputs), dim = 0)

In [92]:
batches.shape

torch.Size([2, 4, 8])

In [ ]:
class CausalAttention(nn.Module): #handles batches of input
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        #create weight matrices
        self.W_q = nn.Linear( d_in, d_out, bias=False )
        self.W_k = nn.Linear( d_in, d_out, bias=False )
        self.W_v = nn.Linear( d_in, d_out, bias=False )
        # include dropout:
        self.dropout = nn.Dropout( dropout )
        # use the following to manage memory effeciently
        self.register_buffer("mask", torch.triu( torch.ones(context_length, context_length), diagonal = 1 ))
        


    # x = embedding vectors or our inputs
    def forward( self, x ):
        b, num_tokens, d_in = x.shape
        # b = batch size 

        queries = self.W_q( x )
        keys = self.W_k( x )
        values = self.W_v( x )
        scores = queries @ keys.transpose(1, 2)
        scores.masked_fill_( self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        weights = torch.softmax( scores / keys.shape[-1]**0.5, dim = -1 )
        weights = self.dropout( weights )
        context = weights @ values
        return context

In [94]:
causal = CausalAttention(d_in=8, d_out=6, context_length=4, dropout=0)

In [95]:
causal(batches)

tensor([[[-0.2801,  0.4566, -0.0922, -0.2678,  0.3117, -0.5622],
         [-0.2931,  0.4365, -0.1526, -0.3974, -0.0170, -0.5062],
         [-0.1752,  0.5183,  0.0115, -0.0622,  0.1382, -0.1060],
         [-0.2423,  0.5500,  0.0260,  0.0285,  0.0581, -0.0713]],

        [[-0.2801,  0.4566, -0.0922, -0.2678,  0.3117, -0.5622],
         [-0.2931,  0.4365, -0.1526, -0.3974, -0.0170, -0.5062],
         [-0.1752,  0.5183,  0.0115, -0.0622,  0.1382, -0.1060],
         [-0.2423,  0.5500,  0.0260,  0.0285,  0.0581, -0.0713]]],
       grad_fn=<UnsafeViewBackward0>)

In [96]:
W_q = nn.Linear( d_in, d_out, bias=False )
W_k = nn.Linear( d_in, d_out, bias=False )
W_v = nn.Linear( d_in, d_out, bias=False )

In [97]:
queries = W_q(batches)
queries

tensor([[[ 0.9158,  0.6004, -0.0566,  0.3280,  0.6197,  0.0381],
         [-0.0827,  0.4487,  0.2674,  0.3950, -0.7260, -0.0990],
         [ 0.7467,  0.6134, -0.0659,  0.1101,  1.4163, -0.4855],
         [-0.1021,  0.3142, -0.4756,  0.1686, -0.8854, -0.0643]],

        [[ 0.9158,  0.6004, -0.0566,  0.3280,  0.6197,  0.0381],
         [-0.0827,  0.4487,  0.2674,  0.3950, -0.7260, -0.0990],
         [ 0.7467,  0.6134, -0.0659,  0.1101,  1.4163, -0.4855],
         [-0.1021,  0.3142, -0.4756,  0.1686, -0.8854, -0.0643]]],
       grad_fn=<UnsafeViewBackward0>)

In [98]:
keys = W_k(batches)
keys

tensor([[[ 0.7923, -0.9866, -1.1296, -0.7488, -0.1404,  0.3901],
         [-0.7893,  0.3488, -0.0692,  0.5706, -0.1268,  0.2835],
         [ 0.9018,  0.2469,  0.7944, -0.7284, -0.2210,  0.3315],
         [-0.4690, -0.5637, -0.3125,  0.8002, -0.4247, -0.2924]],

        [[ 0.7923, -0.9866, -1.1296, -0.7488, -0.1404,  0.3901],
         [-0.7893,  0.3488, -0.0692,  0.5706, -0.1268,  0.2835],
         [ 0.9018,  0.2469,  0.7944, -0.7284, -0.2210,  0.3315],
         [-0.4690, -0.5637, -0.3125,  0.8002, -0.4247, -0.2924]]],
       grad_fn=<UnsafeViewBackward0>)

In [100]:
keys.T

tensor([[[ 0.7923,  0.7923],
         [-0.7893, -0.7893],
         [ 0.9018,  0.9018],
         [-0.4690, -0.4690]],

        [[-0.9866, -0.9866],
         [ 0.3488,  0.3488],
         [ 0.2469,  0.2469],
         [-0.5637, -0.5637]],

        [[-1.1296, -1.1296],
         [-0.0692, -0.0692],
         [ 0.7944,  0.7944],
         [-0.3125, -0.3125]],

        [[-0.7488, -0.7488],
         [ 0.5706,  0.5706],
         [-0.7284, -0.7284],
         [ 0.8002,  0.8002]],

        [[-0.1404, -0.1404],
         [-0.1268, -0.1268],
         [-0.2210, -0.2210],
         [-0.4247, -0.4247]],

        [[ 0.3901,  0.3901],
         [ 0.2835,  0.2835],
         [ 0.3315,  0.3315],
         [-0.2924, -0.2924]]], grad_fn=<PermuteBackward0>)

In [105]:
class MultiHeadAttention( nn.Module ):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
        )
    
    def forward( self, x ):
        return torch.cat( [ head(x) for head in self.heads ], dim=-1 )

In [ ]:
mha = MultiHeadAttention( d_in = 8, d_out = 6, context_length = 4, dropout = 0, num_heads = 3 )

In [107]:
mha_out = mha(batches)

In [108]:
mha_out

tensor([[[-0.0881,  1.4246,  0.6861, -0.7970,  0.5636, -0.0636,  0.4785,
           0.3121, -0.8059, -1.5977,  0.5405,  0.4089, -0.0568,  0.9947,
          -0.1470, -1.2102,  0.2025,  1.4195],
         [-0.0697,  0.5218, -0.1794, -0.3795,  0.2088, -0.3289,  0.5855,
           0.0450, -0.2163, -0.9563,  0.6335, -0.2039,  0.1436,  0.4795,
          -0.0107, -0.6811, -0.0916,  0.5425],
         [-0.0084,  0.7840,  0.5018, -0.3541,  0.2783, -0.1358,  0.1869,
           0.3372, -0.8090, -0.3943,  0.1776, -0.1160, -0.1748,  0.3729,
          -0.1810, -0.5420, -0.2024,  0.6769],
         [ 0.1652, -0.0743, -0.1640, -0.0382,  0.0895, -0.3747,  0.2541,
           0.2574, -0.7839, -0.6654,  0.2674, -0.0360,  0.1747,  0.1574,
           0.2572, -0.2579, -0.1765,  0.3013]],

        [[-0.0881,  1.4246,  0.6861, -0.7970,  0.5636, -0.0636,  0.4785,
           0.3121, -0.8059, -1.5977,  0.5405,  0.4089, -0.0568,  0.9947,
          -0.1470, -1.2102,  0.2025,  1.4195],
         [-0.0697,  0.5218, -0.17

In [109]:
mha_out.shape

torch.Size([2, 4, 18])

In [110]:
#more efficient and better resource allocation
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        
        super().__init__() 
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads 

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) 
        self.dropout = nn.Dropout(dropout)
 
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [111]:
batches.shape

torch.Size([2, 4, 8])

In [112]:
batches

tensor([[[ 0.5442, -0.3583,  0.4284, -0.5506, -0.6946, -2.0262, -1.6451,
          -0.7431],
         [-0.3511, -1.7066, -0.2856,  0.6253, -0.8071,  0.9050, -0.0854,
           0.5278],
         [ 1.1195,  0.3508,  1.0625, -0.1631,  1.9232,  0.1876, -1.9495,
           0.9537],
         [ 0.1406, -0.7225,  0.2274,  0.8747, -2.0024,  2.4018, -0.2076,
          -1.6268]],

        [[ 0.5442, -0.3583,  0.4284, -0.5506, -0.6946, -2.0262, -1.6451,
          -0.7431],
         [-0.3511, -1.7066, -0.2856,  0.6253, -0.8071,  0.9050, -0.0854,
           0.5278],
         [ 1.1195,  0.3508,  1.0625, -0.1631,  1.9232,  0.1876, -1.9495,
           0.9537],
         [ 0.1406, -0.7225,  0.2274,  0.8747, -2.0024,  2.4018, -0.2076,
          -1.6268]]])

In [113]:
batches.view(2,4,2,4)

tensor([[[[ 0.5442, -0.3583,  0.4284, -0.5506],
          [-0.6946, -2.0262, -1.6451, -0.7431]],

         [[-0.3511, -1.7066, -0.2856,  0.6253],
          [-0.8071,  0.9050, -0.0854,  0.5278]],

         [[ 1.1195,  0.3508,  1.0625, -0.1631],
          [ 1.9232,  0.1876, -1.9495,  0.9537]],

         [[ 0.1406, -0.7225,  0.2274,  0.8747],
          [-2.0024,  2.4018, -0.2076, -1.6268]]],


        [[[ 0.5442, -0.3583,  0.4284, -0.5506],
          [-0.6946, -2.0262, -1.6451, -0.7431]],

         [[-0.3511, -1.7066, -0.2856,  0.6253],
          [-0.8071,  0.9050, -0.0854,  0.5278]],

         [[ 1.1195,  0.3508,  1.0625, -0.1631],
          [ 1.9232,  0.1876, -1.9495,  0.9537]],

         [[ 0.1406, -0.7225,  0.2274,  0.8747],
          [-2.0024,  2.4018, -0.2076, -1.6268]]]])

In [ ]:
mha = MultiHeadAttention(d_in = 8, d_out = 6, context_length = 4, dropout = 0, num_heads = 3 )


In [115]:
mha_out = mha(batches)

In [116]:
mha_out

tensor([[[ 0.5516,  0.3211,  0.0186,  0.3213, -0.0403,  0.0474],
         [ 0.2749, -0.1132, -0.1236,  0.4172,  0.0530,  0.0911],
         [ 0.2293, -0.1545, -0.0346,  0.4729, -0.0608,  0.0936],
         [ 0.1880, -0.3267, -0.1679,  0.5925,  0.1006,  0.1268]],

        [[ 0.5516,  0.3211,  0.0186,  0.3213, -0.0403,  0.0474],
         [ 0.2749, -0.1132, -0.1236,  0.4172,  0.0530,  0.0911],
         [ 0.2293, -0.1545, -0.0346,  0.4729, -0.0608,  0.0936],
         [ 0.1880, -0.3267, -0.1679,  0.5925,  0.1006,  0.1268]]],
       grad_fn=<ViewBackward0>)

In [117]:
mha_out.shape

torch.Size([2, 4, 6])